In [130]:
import os
import sys
import shutil
import pandas
import numpy

import ipystata
from ipystata.config import config_stata
config_stata('/usr/local/stata/stata-mp')

from scipy import stats
import statsmodels.api as sm

# import plotly
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# from plotly.graph_objs import Scatter, Figure, Layout, Histogram
# init_notebook_mode(connected=True)
# import PrettyTable as pt



pandas.set_option('display.float_format', lambda x: '%.2f' % x)
pandas.set_option('max_columns', 200)
pandas.set_option('max_rows', 400)
pandas.set_option('max_colwidth', 150)
pandas.set_option('mode.sim_interactive', True)
pandas.set_option('colheader_justify', 'center')

from IPython.display import display, display_markdown, Markdown, HTML
# from IPython.display import display, HTML, Image, IFrame, publish_display_data, display_markdown
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
# pandas.options.display.float_format = '{:.2f}'.format

In [2]:
#https://www.hindawi.com/journals/jeph/2011/435078/

In [56]:
d1 = pandas.read_csv('./data/IzzyLongitudinalWorkingMarch2015.csv')
d2 = pandas.read_csv('./data/AnalysisObservations.csv')

In [72]:
d1.head()
d1.columns

Index(['rsos', 'tsos', 'grip', 'ptiso', 'biodex', 'matlab', 'ntxc', 'calo',
       'matu', 'mvh', 'age'],
      dtype='object')

In [73]:
d2.head()

,id,season,sequence,gender,rsos,tsos,grip,ptiso,Biodex,MatLab,ntxc,matu,mvh,age,Exclusion,NoSOS,NoGrip,Noptiso,Nontx,Nomvh,T,_o,_id,_v,size,_season,omit
0,100,spring,1,boy,3828,3601,nan,113.93,129.10,113.93,711.85,-1.67,105.71,11.75,none,nan,Not taken sequence 1,NaN,NaN,NaN,NaN,1,1,1,4.00,spring,nan
1,100,spring,3,boy,3898,3629,27.00,136.02,138.70,136.02,760.09,-0.71,93.62,12.71,none,nan,NaN,NaN,NaN,NaN,NaN,2,1,2,nan,NaN,nan
2,100,spring,5,boy,3851,3677,37.00,177.05,178.00,177.05,543.73,0.41,98.14,13.83,none,nan,NaN,NaN,NaN,NaN,NaN,3,1,3,nan,NaN,nan
3,100,spring,7,boy,3952,3740,40.50,205.80,205.80,205.80,454.36,1.17,88.14,14.74,none,nan,NaN,NaN,NaN,NaN,NaN,4,1,4,nan,NaN,nan
4,101,spring,1,boy,3682,3603,nan,133.05,139.10,133.05,938.00,-1.63,89.29,11.45,none,nan,Not taken sequence 1,NaN,NaN,NaN,NaN,5,2,1,1.00,spring,nan


In [74]:
dt = d1[['ID', 'Sequence', 'Caloric Intake', 'Height', 'Weight', 'Godin_PA']]
DM = pandas.merge(dt, d2, left_on=['ID', 'Sequence'], right_on=['id', 'sequence'], how='right'); del dt
DM.head()

KeyError: "['ID' 'Sequence' 'Caloric Intake' 'Height' 'Weight' 'Godin_PA'] not in index"

In [75]:
dm = DM.copy()
dm = dm.sort_values(by=['id', 'sequence'])

dm['boys'] = numpy.where(dm.gender=='boy', 1, 0)
dm['girls'] = 1 - dm.boys


dm['trip'] = dm.groupby(['id']).cumcount()+1
dm = dm.rename(columns={'sequence': 'session'})
dm = dm.rename(columns={'Biodex': 'biodex'})
dm = dm.rename(columns={'MatLab': 'matlab'})
dm = dm.rename(columns={'Caloric Intake': 'calo'})
dm = dm.rename(columns={'Height': 'height'})
dm = dm.rename(columns={'Weight': 'weight'})

dm = dm[['id', 'trip', 'session', 'season', 
         'rsos', 'tsos', 
         'grip', 'ptiso','biodex', 'matlab', 
         'ntxc', 'calo', 'mvh', 'Godin_PA'
         'matu', 'age', 'gender', 'boys', 'girls', 'height', 'weight',
         'Exclusion', 'NoSOS', 'NoGrip', 'Noptiso', 'Nontx', 'Nomvh', 'T']]
dm.head()

KeyError: "['Godin_PAmatu'] not in index"

In [76]:
dm.describe()

,ID,Sequence,calo,height,weight,Godin_PA,id,session,rsos,tsos,grip,ptiso,biodex,matlab,ntxc,matu,mvh,age,NoSOS,_o,_id,_v,size,omit,boys,girls,trip
count,406.00,406.00,404.00,406.00,406.00,405.00,406.00,406.00,406.00,406.00,363.00,345.00,406.00,345.00,376.00,406.00,298.00,406.00,0.00,406.00,406.00,406.00,171.00,0.00,406.00,406.00,406.00
mean,333.04,4.19,1593.43,152.15,46.12,73.78,333.04,4.19,3816.08,3687.41,23.43,128.04,131.48,128.04,539.82,-0.86,107.05,11.84,nan,223.08,87.31,1.87,2.40,nan,0.51,0.49,1.86
std,197.62,1.83,479.51,13.61,14.43,39.60,197.62,1.83,100.01,109.42,7.40,56.68,57.73,56.68,263.65,2.02,41.53,1.99,nan,124.68,51.45,0.88,0.92,nan,0.50,0.50,0.88
min,100.00,1.00,469.50,122.80,20.50,0.00,100.00,1.00,3554.00,3367.00,9.00,32.65,41.20,32.65,56.31,-5.30,13.57,7.50,nan,1.00,1.00,1.00,1.00,nan,0.00,0.00,1.00
25%,140.00,3.00,1265.06,141.90,35.23,45.00,140.00,3.00,3747.50,3619.50,18.00,86.12,88.84,86.12,370.82,-2.29,79.47,10.34,nan,118.25,41.00,1.00,2.00,nan,0.00,0.00,1.00
50%,194.00,4.00,1541.21,152.00,44.65,66.00,194.00,4.00,3820.00,3688.00,22.00,115.25,118.65,115.25,514.00,-0.99,99.53,11.84,nan,226.50,92.50,2.00,2.00,nan,1.00,0.00,2.00
75%,531.75,6.00,1848.58,162.17,54.00,93.00,531.75,6.00,3878.00,3757.75,27.00,160.50,161.07,160.50,662.49,0.52,134.03,13.23,nan,331.75,130.75,2.00,3.00,nan,1.00,1.00,2.00
max,583.00,7.00,3624.72,187.60,116.90,234.00,583.00,7.00,4140.00,4015.00,53.00,353.00,353.80,353.00,1754.32,9.66,235.00,17.55,nan,434.00,180.00,4.00,4.00,nan,1.00,1.00,4.00


In [77]:
ds = dm.groupby('session')['rsos', 'tsos', 'grip', 'ptiso','biodex', 'matlab', 'ntxc', 'calo', 'matu', 'mvh', 'age', 'gender'].mean()
d0 = dm[dm['gender']==0].groupby('session')['rsos', 'tsos', 'grip', 'ptiso','biodex', 'matlab', 'ntxc', 'calo', 'matu', 'mvh', 'age'].mean()
d1 = dm[dm['gender']==1].groupby('session')['rsos', 'tsos', 'grip', 'ptiso','biodex', 'matlab', 'ntxc', 'calo', 'matu', 'mvh', 'age'].mean()

t1 = ds.append(d0)
t1 = t1.append(d1)
t1

,rsos,tsos,grip,ptiso,biodex,matlab,ntxc,calo,matu,mvh,age
session,,,,,,,,,,,
1,3798.51,3641.84,nan,109.12,111.64,109.12,741.95,1574.72,-1.85,119.94,11.15
2,3771.03,3657.03,19.13,78.98,101.14,78.98,567.99,1401.74,-1.88,109.12,10.90
3,3790.04,3656.50,20.17,114.10,120.07,114.10,555.63,1639.72,-1.17,121.47,11.29
4,3819.89,3683.83,22.66,127.32,130.79,127.32,527.80,1535.78,-0.94,90.29,11.82
5,3806.17,3681.09,25.15,136.45,142.14,136.45,496.39,1625.33,-0.53,102.19,12.09
6,3858.58,3728.35,24.78,139.68,141.58,139.68,478.24,1559.54,-0.50,112.63,12.25
7,3851.87,3753.53,28.29,161.20,160.05,161.20,462.20,1758.93,0.40,86.40,13.12


In [78]:
def get_st(var):
    temp = 'st_' + var
    dm[temp] = (dm[var] - dm[var].mean()) / dm[var].std()

_vars = ['rsos', 'tsos', 'grip', 'ptiso','biodex', 'matlab', 'ntxc', 'calo', 'matu', 'mvh', 'age', 'height', 'weight', 'Godin_PA']
for v in _vars:
    get_st(v)

dm.to_csv('out/data_analysis.csv', index=False)
dm.head()


,ID,Sequence,calo,height,weight,Godin_PA,id,season,session,gender,rsos,tsos,grip,ptiso,biodex,matlab,ntxc,matu,mvh,age,Exclusion,NoSOS,NoGrip,Noptiso,Nontx,Nomvh,T,_o,_id,_v,size,_season,omit,boys,girls,trip,st_rsos,st_tsos,st_grip,st_ptiso,st_biodex,st_matlab,st_ntxc,st_calo,st_matu,st_mvh,st_age,st_height,st_weight,st_Godin_PA
0,100,1,1572.00,152.00,44.20,37.00,100,spring,1,boy,3828,3601,nan,113.93,129.10,113.93,711.85,-1.67,105.71,11.75,none,nan,Not taken sequence 1,NaN,NaN,NaN,NaN,1,1,1,4.00,spring,nan,1,0,1,0.12,-0.79,nan,-0.25,-0.04,-0.25,0.65,-0.04,-0.40,-0.03,-0.05,-0.01,-0.13,-0.93
1,100,3,2219.01,160.80,48.80,48.00,100,spring,3,boy,3898,3629,27.00,136.02,138.70,136.02,760.09,-0.71,93.62,12.71,none,nan,NaN,NaN,NaN,NaN,NaN,2,1,2,nan,NaN,nan,1,0,2,0.82,-0.53,0.48,0.14,0.13,0.14,0.84,1.30,0.07,-0.32,0.44,0.64,0.19,-0.65
2,100,5,2599.83,173.30,63.50,44.00,100,spring,5,boy,3851,3677,37.00,177.05,178.00,177.05,543.73,0.41,98.14,13.83,none,nan,NaN,NaN,NaN,NaN,NaN,3,1,3,nan,NaN,nan,1,0,3,0.35,-0.10,1.83,0.86,0.81,0.86,0.01,2.10,0.63,-0.21,1.00,1.55,1.20,-0.75
3,100,7,2482.12,177.40,67.30,66.00,100,spring,7,boy,3952,3740,40.50,205.80,205.80,205.80,454.36,1.17,88.14,14.74,none,nan,NaN,NaN,NaN,NaN,NaN,4,1,4,nan,NaN,nan,1,0,4,1.36,0.48,2.31,1.37,1.29,1.37,-0.32,1.85,1.00,-0.46,1.46,1.85,1.47,-0.20
4,101,1,1549.99,158.10,47.30,25.00,101,spring,1,boy,3682,3603,nan,133.05,139.10,133.05,938.00,-1.63,89.29,11.45,none,nan,Not taken sequence 1,NaN,NaN,NaN,NaN,5,2,1,1.00,spring,nan,1,0,1,-1.34,-0.77,nan,0.09,0.13,0.09,1.51,-0.09,-0.38,-0.43,-0.20,0.44,0.08,-1.23


In [79]:
# pandas.set_option('display.float_format', lambda x: '%.2f' % x)
de = dm[['id', 'trip', 
         'rsos', 'tsos', 
         'grip', 'biodex', 'ntxc', 'calo', 'mvh', 
         'matu', 'gender', 'boys', 'girls']]

dt = de.copy()
dt = dt.drop(columns=['gender'])
dt['gender'] = 'All'

ds = de.copy()
ds = dt.append(ds)

ds = ds.set_index('gender')
ds = ds.rename(columns={'rsos': 'Radial SOS'})
ds = ds.rename(columns={'tsos': 'tibial SOS'})
ds = ds.rename(columns={'grip': 'grip strength'})
ds = ds.rename(columns={'biodex': 'knee extensors'})
ds = ds.rename(columns={'matu': 'maturity offset'})
ds = ds.rename(columns={'ntxc': 'ntx creatine'})
ds = ds.rename(columns={'calo': 'caloric intake'})
ds = ds.rename(columns={'mvh': 'MVH'})

dt = ds.groupby('gender').agg(['mean','std', 'count'])
dt = dt.rename(columns={'mean':'Mean'})
dt = dt.rename(columns={'std':'Standard deviation'})
dt = dt.rename(columns={'count':'Observations'})

dt.index.name = ''
dt = dt.T
dt.to_csv('table1.csv'); del ds

In [80]:
%%stata --data de 

xtset id trip
xtdescribe


       panel variable:  id (unbalanced)
        time variable:  trip, 1 to 4
                delta:  1 unit

      id:  100, 101, ..., 583                                n =        173
    trip:  1, 2, ..., 4                                      T =          4
           Delta(trip) = 1 unit
           Span(trip)  = 4 periods
           (id*trip uniquely identifies each observation)

Distribution of T_i:   min      5%     25%       50%       75%     95%     max
                         1       1       2         2         3       4       4

     Freq.  Percent    Cum. |  Pattern
 ---------------------------+---------
       65     37.57   37.57 |  111.
       52     30.06   67.63 |  11..
       39     22.54   90.17 |  1...
       17      9.83  100.00 |  1111
 ---------------------------+---------
      173    100.00         |  XXXX


In [81]:
%%stata --data de

xtset id trip
xtsum rsos tsos
swilk rsos tsos
ttest rsos == tsos

*twoway (hist rsos, frac lcolor(gs12) fcolor(gs12)) (hist tsos, frac fcolor(none) lcolor(red)), legend(off) xtitle("rsos (red: tsos)")



       panel variable:  id (unbalanced)
        time variable:  trip, 1 to 4
                delta:  1 unit

Variable         |      Mean   Std. Dev.       Min        Max |    Observations
-----------------+--------------------------------------------+----------------
rsos     overall |  3816.084   100.0084       3554       4140 |     N =     406
         between |             86.03224       3622       4119 |     n =     173
         within  |             51.45185   3496.084   4012.084 | T-bar = 2.34682
                 |                                            |
tsos     overall |  3687.411   109.4166       3367       4015 |     N =     406
         between |             96.74594       3395     3925.5 |     n =     173
         within  |             49.81649   3501.161   3861.161 | T-bar = 2.34682

                   Shapiro-Wilk W test for normal data

    Variable |        Obs       W           V         z       Prob>z
-------------+----------------------------------------------

In [82]:
%%stata --data de

xtset id trip

bys gender: xtsum rsos tsos

di "radial SOS****************"
ttest rsos, by(gender)

di "tibial SOS****************"

ttest tsos, by(gender)



       panel variable:  id (unbalanced)
        time variable:  trip, 1 to 4
                delta:  1 unit

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
-> gender = boy

Variable         |      Mean   Std. Dev.       Min        Max |    Observations
-----------------+--------------------------------------------+----------------
rsos     overall |  3806.622   96.94986       3554       4074 |     N =     209
         between |             83.66979       3641       4004 |     n =      93
         within  |              45.0813   3486.622   4002.622 | T-bar = 2.24731
                 |                                            |
tsos     overall |  3676.459   101.3152       3367       4015 |     N =     209
         between |             93.20911       3395     3925.5 |     n =      93
         within  |             40.71837   3523.4

In [83]:
# %%stata --data de

# xtset id trip
# xtsum grip biodex ntxc calo mvh matu 
# xtsum grip biodex ntxc calo mvh matu , by(gender)

The repeated sampling of participants resulted in a hierarchical data set that consisted of within (level 1) and between (level 2) measurement variations. Table 1 outlines the variables used in the analysis, their means, and their standard deviations within and between participants. Shapiro-Wilk's tests did not reject the null hypothesis that the radial and tibial bone property distributions are normally distribution. Radial SOS measured an average of 129 points greater than tibial SOS (p=0.00). Girls, on average, has greater bone property measurements than boys; radial SOS by 19 (p=0.0495) and tibial SOS 22 points (0.0376). 

The bone property variances exhibited two distinct characteristics. One is that notable between and within variances existed. Another is that greater variation was exhibited across participants (i.e., between) than across time (i.e., within). The standard deviation of radial SOS measurements across participants (i.e., between) was 86 m/s compared to 51 m/s. The across and with participant standard deviation of tibial SOS was 96 and 50 m/s, respectively. Each independent variable also had a greater between compared to within variance to suggest that a fixed-participant estimation procedure may lead to considerable efficiency loss and a random-participant model is appropriate.


A multivariate multilevel structural equation model (i.e., SEM) was used to describe bone properties. In brief, SEM is a multivariate statistical technique used to estimate a system of equations and test hypotheses about the relationships among variables. To do so, SEM explicates the direct relationships between observed variables and the covariance relationships between unobserved variables. Models were constructed using Stata 14’s Generalized Structural Equation package (see Kaplan, 2008; Kline, 2015 for an overview of structural equation modeling). 

The functional model of bone development provides the conceptual framework to model bone property changes. This model postulates that specific modulators influence muscle strength, which in turn, along with osteoblasts/osteoclasts related cellular modulators, influence bone properties (see Figure 1). The general SEM to describe bone property changes can be expressed as

$ y_{i,t} =  \beta_0 + \beta_1 m_{i,t} + \beta_2 {x_2}_{i,t} + .... + \beta_k {x_k}_{i,t} + \gamma_i + \epsilon_{i,t}$

$ m_{i,t} =  \alpha_0 + \alpha_1 {z_1}_{i,t} + .... + \beta_m {z_m}_{i,t} + \mu_{i,t}$

$ \Sigma = 
\begin{bmatrix}
 \sigma^2_{\epsilon} & \sigma_{\epsilon, \mu}  \\
 \sigma_{\mu, \epsilon} & \sigma^2_{\mu}  \\
\end{bmatrix} $

The $y_{i,t}$ term denotes the $i^{th}$ participant's bone property measurement on the $t^{th}$ occasion. The $m_{i,t}$ term donotes his or her muscle strength, and the ${x_k}_{i,t}$ term donates modulators and control variables. The $\alpha$, and $\beta$ terms denote the unknown fixed parameters, the $\epsilon_{i,t}$ and $\mu_{i,t}$ terms denote the unobserved within-participant (level-1) residuals, and the $\gamma_{i}$ term denotes the unobserved random participant-effects (level 2). The random participant-effects are assumed to be normally distributed with a mean of zero and independent of the muscle strength ($m$) and modulator variables (i.e., $x$). The $\Sigma$ term donates the variance-covariance matrix of the residual terms.

Figure 2 dipicts the bivariate multilevel medidated model implemented. , with obsevered radial and tibial properties. The $\Sigma^W$ and $\Sigma^U$ term denotes the covariances among the within-participant random bone and muscle components. 



In [84]:
dm.head()

,ID,Sequence,calo,height,weight,Godin_PA,id,season,session,gender,rsos,tsos,grip,ptiso,biodex,matlab,ntxc,matu,mvh,age,Exclusion,NoSOS,NoGrip,Noptiso,Nontx,Nomvh,T,_o,_id,_v,size,_season,omit,boys,girls,trip,st_rsos,st_tsos,st_grip,st_ptiso,st_biodex,st_matlab,st_ntxc,st_calo,st_matu,st_mvh,st_age,st_height,st_weight,st_Godin_PA
0,100,1,1572.00,152.00,44.20,37.00,100,spring,1,boy,3828,3601,nan,113.93,129.10,113.93,711.85,-1.67,105.71,11.75,none,nan,Not taken sequence 1,NaN,NaN,NaN,NaN,1,1,1,4.00,spring,nan,1,0,1,0.12,-0.79,nan,-0.25,-0.04,-0.25,0.65,-0.04,-0.40,-0.03,-0.05,-0.01,-0.13,-0.93
1,100,3,2219.01,160.80,48.80,48.00,100,spring,3,boy,3898,3629,27.00,136.02,138.70,136.02,760.09,-0.71,93.62,12.71,none,nan,NaN,NaN,NaN,NaN,NaN,2,1,2,nan,NaN,nan,1,0,2,0.82,-0.53,0.48,0.14,0.13,0.14,0.84,1.30,0.07,-0.32,0.44,0.64,0.19,-0.65
2,100,5,2599.83,173.30,63.50,44.00,100,spring,5,boy,3851,3677,37.00,177.05,178.00,177.05,543.73,0.41,98.14,13.83,none,nan,NaN,NaN,NaN,NaN,NaN,3,1,3,nan,NaN,nan,1,0,3,0.35,-0.10,1.83,0.86,0.81,0.86,0.01,2.10,0.63,-0.21,1.00,1.55,1.20,-0.75
3,100,7,2482.12,177.40,67.30,66.00,100,spring,7,boy,3952,3740,40.50,205.80,205.80,205.80,454.36,1.17,88.14,14.74,none,nan,NaN,NaN,NaN,NaN,NaN,4,1,4,nan,NaN,nan,1,0,4,1.36,0.48,2.31,1.37,1.29,1.37,-0.32,1.85,1.00,-0.46,1.46,1.85,1.47,-0.20
4,101,1,1549.99,158.10,47.30,25.00,101,spring,1,boy,3682,3603,nan,133.05,139.10,133.05,938.00,-1.63,89.29,11.45,none,nan,Not taken sequence 1,NaN,NaN,NaN,NaN,5,2,1,1.00,spring,nan,1,0,1,-1.34,-0.77,nan,0.09,0.13,0.09,1.51,-0.09,-0.38,-0.43,-0.20,0.44,0.08,-1.23


In [113]:
%%stata 

log query
log close

qui log using example, replace smcl

/***
Writing dynamic text in MarkDoc
===============================

Use the `txt` command to write dynamic text in **MarkDoc** as shown below:
***/

//OFF
sysuse auto, clear
summarize price
//ON

txt "mean of Price variable is " r(mean) " and SD is " %9.3f r(sd)    ///
    ". Note that you only need to put the string in double quotes, if you " ///
    "wish to display **Scalars** within the text. Instead, if you include " ///
    "local or global **Macros**, you can just type any text without "       ///
    "bothering with the double quotations."

qui log close

markdoc example, replace noisily export(html) install title("How to write text in MarkDoc") 

In [131]:
%%stata --data dm 

log close
qui log using results, replace smcl

//OFF
rename biodex biod
gen lng = ln(grip)
gen lnb = ln(biod)
gen lnr = ln(rsos)
gen lnt = ln(tsos)

rename st_rsos r
rename st_tsos t
rename st_grip g
rename st_biod b
rename st_matu m
rename st_ntxc n
rename st_calo c
rename st_mvh v

gen rb = boys * r
gen rg = girls * r

set cformat %9.4f

#delimit ;

eststo m1: gsem
    (r <- g@k1   _cons@kk)
    (t <- b@k1 v _cons@kk)
    (r <- m@k2 n@k3  M1[id]@1 _cons@cc) 
    (g b <- m@k4 c@k5 M2[id]@1), 
    covstruct(_lexogenous, diagonal)   
    nocapslatent latent(M1 M2) 
    cov(e.r*e.t) cov(e.g*e.b) cov(M1[id]*M2[id])
    nohead nolog 
    ;
    
#delimit cr
//ON
esttab m1 
qui log close

markdoc results, replace export(html)  title("Estimation results") 

In [142]:
HTML("results.html")



In [ ]:
# %load results.md
Estimation results
==================

          . esttab m1 

          ----------------------------
                                (1)   
                                  r   
          ----------------------------
          r                           
          g                   0.333***
                             (6.80)   

          m                  0.0913   
                             (1.71)   

          n                  -0.191***
                            (-4.30)   

          M1[id]                  1   
                                (.)   

          _cons             -0.0136   
                            (-0.22)   
          ----------------------------
          t                           
          b                   0.333***
                             (6.80)   

          v                  -0.107*  
                            (-2.07)   

          _cons             -0.0825   
                            (-1.60)   
          ----------------------------
          g                           
          m                   0.628***
                            (19.39)   

          c                  0.0815** 
                             (3.02)   

          M2[id]                  1   
                                (.)   

          _cons            -0.00623   
                            (-0.12)   
          ----------------------------
          b                           
          m                   0.628***
                            (19.39)   

          c                  0.0815** 
                             (3.02)   

          M2[id]                  1   
                                (.)   

          _cons              0.0304   
                             (0.60)   
          ----------------------------
          var(M1[id])                 
          _cons               0.444***
                             (6.40)   
          ----------------------------
          var(M2[id])                 
          _cons               0.359***
                             (7.63)   
          ----------------------------
          cov(M2[id]~)                
          _cons             -0.0725   
                            (-1.64)   
          ----------------------------
          var(e.r)                    
          _cons               0.318***
                             (7.87)   
          ----------------------------
          var(e.t)                    
          _cons               0.805***
                            (12.14)   
          ----------------------------
          var(e.g)                    
          _cons               0.283***
                            (10.01)   
          ----------------------------
          var(e.b)                    
          _cons               0.159***
                             (9.43)   
          ----------------------------
          cov(e.t,e.r)                
          _cons               0.256***
                             (4.63)   
          ----------------------------
          cov(e.b,e.g)                
          _cons              0.0379** 
                             (2.72)   
          ----------------------------
          N                     404   
          ----------------------------
          t statistics in parentheses
          * p<0.05, ** p<0.01, *** p<0.001


In [118]:
%%stata --data dm

quietly  set processors 20
quietly  xtset id trip

rename biodex biod

global Z "matu ntxc"
global W "matu calo"

gsem (rsos <- grip $Z M[id] ) (grip <-  $W ) ///
, covstruct(_lexogenous, diagonal) nocapslatent latent(M L) ///
noheader nodvheader vsquish nocnsreport noomitted nolog nohead ///
cov(M[id]*L[id])
estimates store m1


> , covstruct(_lexogenous, diagonal) nocapslatent latent(M L) ///
> noheader nodvheader vsquish nocnsreport noomitted nolog nohead ///
> cov(M[id]*L[id])
option nohead not allowed
r(198);

end of do-file

r(198);


In [ ]:
%%stata --data dm

quietly  set processors 20
quietly  xtset id trip

rename biodex biod

sum rsos tsos
sum grip biod 

global Z "matu ntxc"
global W "matu calo"

quietly {
    gsem (rsos <- grip $Z M[id] ) (grip <-  $W L) ///
, covstruct(_lexogenous, diagonal) nocapslatent latent(M L) ///
noheader nodvheader vsquish nocnsreport noomitted nolog nohead ///
cov(M[id]*L[id])
estimates store m1
}

gsem, noheader

In [ ]:
%%stata --data dm
set processors 20
xtset id trip

gen biod = biodex



gsem (rsos <- grip  ntxc M@a) (grip <- matu calo  M@a) ///
, covstruct(_lexogenous, diagonal) nocapslatent latent(M) var(M@1) ///
noheader nodvheader vsquish nocnsreport noomitted 

In [ ]:
%%stata --data dm
set processors 20
xtset id trip

global Z "matu ntxc"
global W "matu calo"

gen biod = biodex

*sem (rsos <- grip matu ntxc) (grip <- matu calo)
* sem (rsos <- grip matu ntxc) (L -> grip)

quietly gsem (rsos <- grip  ntxc M[id] ) (grip <- matu calo L[id]) ///
, covstruct(_lexogenous, diagonal) nocapslatent latent(M L) ///
noheader nodvheader vsquish nocnsreport noomitted ///
cov(e.rsos*e.grip) cov(M[id]*L[id])
estimates store m1

gsem, noheader


In [ ]:
%%stata --data dm 
quietly set processors 20
quietly xtset id trip

display "Mediated"
quietly gsem ///
(rsos <- grip matu ntxc ) (grip <- matu calo) /// 
, covstruct(_lexogenous, diagonal) nocapslatent latent(M) ///
noheader nodvheader vsquish nocnsreport noomitted
estimates store m1

gsem, noheader



In [ ]:
%%stata --data dm 
set processors 20
xtset id trip

display "Mediated"
gsem ///
(rsos <- grip matu ntxc M[id]) (grip <- matu calo) /// ///
, covstruct(_lexogenous, diagonal) nocapslatent latent(M)
estimates store m1


display "Mediated"
gsem ///
(rsos <- grip matu ntxc ) (grip <- matu calo M[id]) /// ///
, covstruct(_lexogenous, diagonal) nocapslatent latent(M)
estimates store m2

estimates stats m1 m2


In [ ]:
%%stata --data dm
set processors 20
xtset id trip

gsem (rsos <- grip  ntxc L) (grip <- matu calo L), nolog latent(L)


In [ ]:
%%stata --data dm
set processors 20
xtset id trip

global Z "matu ntxc"
global W "matu calo"

gen biod = biodex


gsem ///
(rsos <- grip $Z M[id]@1) (grip <-$W) ///
(tsos <- biod $Z) (biod <-$W) ///
, covstruct(_lexogenous, diagonal) latent(M) ///
cov(e.rsos*e.tsos) cov(e.grip*e.biod)

In [ ]:
%%stata --data dm 
set processors 20
xtset id trip

gsem ///
(rad <- grp mat ntx M[id]@1) ///
(tib <- tor mat ntx mvh M[id]@1) /// 
, covstruct(_lexogenous, diagonal) latent(M) ///
cov(e.rad*e.tib)

In [ ]:
%%stata --data dm 
set processors 20
xtset id 

xtsum rsos
xtreg rsos, re mle

*xtivreg rsos ntx matu (grip = matu cal), fe
loneway rsos id


In [ ]:
%%stata --data dm

swilk rsos tsos

display "*******************************"
display "t-test radial SOS across gender"
ttest rsos, by(gender)

display "t-test tibial SOS across gender"
ttest tsos, by(gender)

ttest rsos==tsos

In [ ]:
%%stata --data dm

gen lngrip = log(grip)
gen lnbiodex = log(biodex)


swilk grip biodex 
swilk lngrip lnbiodex

swilk matu ntxc calo mvh


In [ ]:
%%stata --data dm --graph

gen lngrip = log(grip)
gen lnbiodex = log(biodex)

histogram lnbiodex

In [ ]:
%%stata --data dm --graph

twoway (hist rsos, frac lcolor(gs12) fcolor(gs12)) (hist tsos, frac fcolor(none) lcolor(red)), legend(off) xtitle("rsos (red: tsos)")


In [ ]:
%%stata --data dm 
set processors 20
xtset id trip

xtsum rsos tsos grip biodex

xttab rsos

In [ ]:
dr = dm[['id', 'trip', 'rsos', 'grip', 'matu', 'ntxc', 'calo']]
dr = dr.set_index(['id', 'trip'])

d1 = dr.shift(1)
# dr['lnrsos'] = numpy.log(dr.rsos)
# # dr['r1'] = dr.lnrsos.shift(1)
# dr['rp'] = dr.lnrsos - dr.lnrsos.shift(1)

# dr['rsos1'] = dr.rsos.shift(1)
# dr['rsosd'] = (dr.rsos - dr.rsos1) 
# dr['rsosp'] = dr.rsosd / dr.rsos1
# dr['rsosp1'] = dr.rsos / dr.rsos.shift(1) -1

dp = dr / d1 - 1
# dp.groupby('trip').mean()

In [ ]:
rsos grip matu ntxc

In [ ]:
%%stata --data dm 
set processors 20
xtset id trip



ivregress 2sls st_rsos  st_ntx (st_grip = st_height st_weight st_cal ), first 
estat endogenous



In [ ]:
%%stata --data dm 
set processors 20
xtset id trip


display "Mediated"
sem (st_rsos <- st_grip st_matu st_ntx) (st_grip <- st_matu) , nolog nocapslatent

In [ ]:
%%stata --data dm 
set processors 20
xtset id trip

display "******************************************************************"
sureg (rsos grip matu ntxc) (tsos ptiso matu ntxc mvh), isure

display "******************************************************************"
sureg (rsos grip matu ntxc) (tsos biodex matu ntxc mvh), isure

display "******************************************************************"
sureg (st_rsos st_grip st_matu st_ntxc) (st_tsos st_ptiso st_matu st_ntxc st_mvh), isure

display "******************************************************************"
sureg (st_rsos st_grip st_matu st_ntxc) (st_tsos st_bio st_matu st_ntxc st_mvh), isure



In [ ]:
%%stata --data dm 
set processors 20
xtset id trip

sem ///
(rad <- grp mat ntx) ///
(tib <- tor mat ntx mvh) /// 
, covstruct(_lexogenous, diagonal) nocapslatent ///
cov(e.rad*e.tib) 
    

display "Mediated"
sem ///
(rad <- grp mat ntx) (grp <- mat cal) ///
(tib <- tor mat ntx mvh)  (tor <- mat cal)  /// 
, covstruct(_lexogenous, diagonal) nocapslatent ///
cov(e.rad*e.tib) 

In [ ]:
%%stata --data dm 
set processors 20
xtset id trip

gsem ///
(rad <- grp mat ntx M[id]@1) ///
(tib <- tor mat ntx mvh M[id]@1) /// 
, covstruct(_lexogenous, diagonal) latent(M) ///
cov(e.rad*e.tib)


display "Mediated"
gsem ///
(rad <- grp mat ntx M[id]@1) (grp <- mat cal) ///
(tib <- tor mat ntx mvh M[id]@1) (tor <- mat cal)  /// 
, covstruct(_lexogenous, diagonal) nocapslatent  latent(M) ///
cov(e.rad*e.tib) 

display "**********"

gsem ///
(rad <- grp mat ntx M[id]@1) (grp <- mat cal) ///
(tib <- tor mat ntx mvh M[id]@1) (tor <- mat cal)  /// 
, covstruct(_lexogenous, diagonal) nocapslatent  latent(M) ///
cov(e.rad*e.tib) cov(e.grp*e.tor)


In [ ]:
%%stata --data dm 
set processors 20
xtset id trip

gsem ///
(rad <- grp mat ntx M[id]@1) (grp <- mat cal) ///
(tib <- tor mat ntx mvh M[id]@1) (tor <- mat cal)  /// 
, covstruct(_lexogenous, diagonal) nocapslatent  latent(M) ///
cov(e.rad*e.tib) cov(e.grp*e.tor)   

lincom _b[grp:mat] - _b[tor:mat]
lincom _b[grp:cal] - _b[tor:cal]

lincom _b[rad:grp] - _b[tib:tor]
lincom _b[rad:mat] - _b[tib:mat]
lincom _b[rad:ntx] - _b[tib:ntx]

In [ ]:
gsem ///
    (M1[id] griphs matus ntxhs godin  -> rsoshs, ) /// 
    (M2[id] torqhs matus ntxhs mvhhs  -> tsoshs, ) /// 
    (cinths matus  -> griphs,)  ///
    (cinths matus  -> torqhs,)  ///
     , covstruct(_lexogenous, diagonal) nocapslatent ///
      latent(M1 M2)   ///
      cov(e.rsoshs*e.tsoshs)  cov(e.torqhs*e.griphs)  ///
      cov(e.rsoshs*e.griphs)  cov(e.tsoshs*e.torqhs) ///
      cov(M1[id]*M2[id])  
        
estimates save "./out/table2_b"

lincom [griphs]matus - [torqhs]matus
lincom [griphs]cinths - [torqhs]cinths

lincom [rsoshs]griphs - [tsoshs]torqhs
lincom [rsoshs]matus - [tsoshs]matus
lincom [rsoshs]ntxhs - [tsoshs]ntxhs
lincom [rsoshs]paqps - [tsoshs]mvhhs